In [ ]:
# Run the pre-processing and the auxiliary functions
%run 'pre-processing.ipynb'
%run 'auxiliary.ipynb'
from ipynb.fs.full.auxiliary import *

In [ ]:
freebase_id = '/m/02w7gg'
translate_freebase(freebase_id)

In [ ]:
id = '330'
root = parse_nlp(id) 

for e in root.iter('word') :
    print(e.text)